# Лабораторная работа №4 “Нейронные сети”

In [424]:
import matplotlib.pyplot as plt
import matplotlib.colors as clr
from scipy import optimize
from scipy.io import loadmat
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
import pandas as pd

## Task 1

Набор данных ex4data1.mat (такой же, как в лабораторной работе №2)
представляет собой файл формата *.mat (т.е. сохраненного из Matlab). Набор
содержит 5000 изображений 20x20 в оттенках серого. Каждый пиксель
представляет собой значение яркости (вещественное число). Каждое
изображение сохранено в виде вектора из 400 элементов. В результате
загрузки набора данных должна быть получена матрица 5000x400. Далее
расположены метки классов изображений от 1 до 9 (соответствуют цифрам
от 1 до 9), а также 10 (соответствует цифре 0).

1. Загрузите данные ex4data1.mat из файла.

In [439]:
task1_file_name = "Data\Lab 4\ex4data1.mat"
mat = loadmat(task1_file_name)  # load mat-file
X = mat['X']
y = mat['y']

2. Загрузите веса нейронной сети из файла ex4weights.mat, который
содержит две матрицы Θ<sup>(1)</sup>(25, 401) и Θ<sup>(2)</sup>(10, 26). Какова структура полученной нейронной сети?

In [440]:
task1_weights_file_name = "Data\Lab 4\ex4weights.mat"
mat = loadmat(task1_weights_file_name)  # load mat-file
Theta1 = mat['Theta1']
Theta2 = mat['Theta2']

3. Реализуйте функцию прямого распространения с сигмоидом в качестве функции активации.

In [427]:
sigmoid = lambda z: 1 / (1 + np.exp(-z))
perceptron = lambda t, x, func: func(np.dot(t, x))

In [428]:
X.shape, y.shape, Theta1.shape, Theta2.shape

((5000, 400), (5000, 1), (25, 401), (10, 26))

In [429]:
def layer_perform(weights, input_data, func):
    result = np.zeros((weights.shape[0], input_data.shape[1]))
    z = np.dot(weights, input_data)
    a = perceptron(weights, input_data, func)
    return z, a

In [635]:
zero_layer = np.hstack((np.ones((X.shape[0], 1)), X)).T
z2, first_layer = layer_perform(Theta1, zero_layer, sigmoid)
first_layer = np.hstack((np.ones((first_layer.shape[1], 1)), first_layer.T)).T
z3, second_layer = layer_perform(Theta2, first_layer, sigmoid)

In [636]:
pred = np.argmax(second_layer.T, axis = 1) + 1 
pred.shape

(5000,)

4. Вычислите процент правильных классификаций на обучающей выборке. Сравните полученный результат с логистической регрессией.

In [637]:
np.mean(pred.flatten() == y.flatten()) * 100

97.52

5. Перекодируйте исходные метки классов по схеме one-hot.

In [14]:
k = 10
class_matrix = np.eye(k)
class_model = {}
for i in range(k):
    class_model[i+1] = class_matrix[:, i].reshape(1, k)

In [15]:
class_model

{1: array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 2: array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 3: array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]]),
 4: array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
 5: array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]),
 6: array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]),
 7: array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]]),
 8: array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]]),
 9: array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 10: array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])}

In [16]:
y_in_hot = []
for item in y.T.tolist()[0]:
    y_in_hot.append(class_model[item])
y_in_hot = np.array(y_in_hot)
y_in_hot = y_in_hot.reshape(5000, 10)
y_in_hot.shape

(5000, 10)

6. Реализуйте функцию стоимости для данной нейронной сети.

In [742]:
lost_func = lambda t, x, y: (-1 / m) * (
                                 np.dot(y.T,np.log(sigmoid(np.dot(t.T, x)).T)) 
                                 + 
                                 np.dot((1 - y).T, np.log(1 - sigmoid(np.dot(t.T, x)).T))
                                )

7. Добавьте L2-регуляризацию в функцию стоимости.

In [87]:
g = lambda z: 1 / (1 + np.exp(-z))
lost_func = lambda t, x, y, l: (-1 / m) * (
                                 np.dot(y.T,np.log(g(np.dot(t.T, x)).T)) 
                                 + 
                                 np.dot((1 - y).T, np.log(1 - g(np.dot(t.T, x)).T))
                                ) + l*np.sum(np.square(t))/(2*m)

8. Реализуйте функцию вычисления производной для функции активации

In [17]:
derivative_func = lambda z, func: func(z) * (1-func(z))

9. Инициализируйте веса небольшими случайными числами.

In [638]:
# (b - a) * random_sample() + a [-1; 1)
T1 = 4 * np.random.random_sample(Theta1.shape) - 2
T2 = 4 * np.random.random_sample(Theta2.shape) - 2

10.Реализуйте алгоритм обратного распространения ошибки для данной
конфигурации сети

In [740]:
T = [T1, T2, None]
iteration = 1

In [741]:
zero_layer = np.hstack((np.ones((X.shape[0], 1)), X)).T
alpha = 0.1
a = [zero_layer, None, None]
z = [None, None, None]
# T = [T1, T2, None]
beta = [None, None, None]
delta = [np.zeros(T1.shape), np.zeros(T2.shape), None]
D = [np.zeros(T1.shape), np.zeros(T2.shape), None]
m = X.shape[0]
L=3
for num_layer in range(1, L):
    z[num_layer], a[num_layer] = layer_perform(T[num_layer-1], a[num_layer-1], sigmoid)
    if num_layer < L - 1:
        a[num_layer] = np.hstack((np.ones((a[num_layer].shape[1], 1)), a[num_layer].T)).T
        z[num_layer] = np.hstack((np.ones((z[num_layer].shape[1], 1)), z[num_layer].T)).T
beta[-1] = np.abs(a[-1]-y_in_hot.T)
print(a[-1])
print(np.mean(beta[-1]))
for num_layer in reversed(range(1, L-1)):
    beta[num_layer] = np.dot(T[num_layer].T, beta[num_layer+1]) * derivative_func(z[num_layer], sigmoid)
for num_layer in range(L-1):
    if num_layer < L-2:
        delta[num_layer] = delta[num_layer] + np.dot(b[num_layer+1][:-1,:], a[num_layer].T)
    else:
        delta[num_layer] = delta[num_layer] + np.dot(b[num_layer+1], a[num_layer].T)
for num_layer in range(L-1):
    D[num_layer] = delta[num_layer] / m
for num_layer in range(L-1):
    T[num_layer] = T[num_layer] - alpha * D[num_layer]
iteration += 1
print(iteration)
pred = np.argmax(a[-1].T, axis = 1) 
print(np.mean(pred.flatten() == y.flatten()))

[[0.1993106  0.36892792 0.74291086 ... 0.17514858 0.18414321 0.03360167]
 [0.98698918 0.95248829 0.47492096 ... 0.98650254 0.7730153  0.67289879]
 [0.28442474 0.18917908 0.00626088 ... 0.27782049 0.66294811 0.04954411]
 ...
 [0.88008896 0.77080539 0.99827166 ... 0.80278778 0.56444731 0.94004053]
 [0.09660648 0.13414081 0.0047288  ... 0.02015726 0.00460698 0.01193369]
 [0.174368   0.17245953 0.00104751 ... 0.00137724 0.0010163  0.03051063]]
0.3719924716653812
2
0.0902


In [99]:
np.dot(beta[1][:-1,:], a[0].T).shape, delta[0].shape

((25, 401), (25, 401))

11.Для того, чтобы удостоверится в правильности вычисленных значений
градиентов используйте метод проверки градиента с параметром ε = 10<sup>-4</sup>

In [ ]:
for num_layer in range(L-1):
    t = T[num_layer]
    for i in range(t.shape[1]):
        for j in range(t.shape[0]):
            

12.Добавьте L2-регуляризацию в процесс вычисления градиентов.

13.Проверьте полученные значения градиента.

14.Обучите нейронную сеть с использованием градиентного спуска или
других более эффективных методов оптимизации.

15.Вычислите процент правильных классификаций на обучающей
выборке.

16.Визуализируйте скрытый слой обученной сети.

17.Подберите параметр регуляризации. Как меняются изображения на
скрытом слое в зависимости от данного параметра?